We consider example 1.1 of [H00].
There are two modes:
* Off: the heater is off, the temperature falls according to the flow condition $\dot{x} = -0.1x$.
* On: the heater is on, the temperature rises according to the flow condition $\dot{x} = 5 - 0.1x$.

The heater may turn on as soon as $x < 19$ and cannot only be off while $x \ge 18$.
Similarly, the heater may turn off as soon as $x > 21$ and can only be on while $x \le 22$.

[H00] Henzinger, T. A.
*The theory of hybrid automata*.
In Verification of Digital and Hybrid Systems, **2000**, pp. 265-292. Springer, Berlin, Heidelberg.

In [Polyhedra](https://github.com/JuliaPolyhedra/Polyhedra.jl), polyhedra are represented as the intersection of halfspaces. Each halfspace is represented by the vector $a$ and the scalar $\beta$ of the inequality $a^\top x \le \beta$.
The inequality $x \ge 18$ is hence rewritten $-x \le -18$.
Even if there is only one halfspace, `intersect` need to be called to transform the hyperplane into a H-representation of a polyhedron.

In [4]:
using Polyhedra
domain_off      = intersect(HyperPlane([-1.0], -18))  # -x ≤ -18 ≡ x ≥ 18
domain_on       = intersect(HyperPlane([ 1.0],  22))  #  x ≤  22
guard_off_to_on = intersect(HyperPlane([-1.0], -19))  # -x ≤ -19 ≡ x ≥ 19
guard_on_to_off = intersect(HyperPlane([ 1.0],  21)); #  x ≤  21

Some polyhedral operation require the H-representation to be turned into a polyhedron linked with a library that will be used to apply the operation and compute the resulting polyhedron.
Depending on the application, the H-representation constructed in the above cell may need to be transformed in to polyhedron as follows (we use the CDD library in this example but other are availabe, see [here](https://juliapolyhedra.github.io/) for a list):

In [6]:
using CDDLib
lib = CDDLib.Library()
p_domain_off      = polyhedron(domain_off,      lib)
p_domain_on       = polyhedron(domain_on,       lib)
p_guard_off_to_on = polyhedron(guard_off_to_on, lib)
p_guard_on_to_off = polyhedron(guard_on_to_off, lib);

The mode contains both the guard as respectively the dynamic and the continuous state constraint set.

In [ ]:
using MathematicalSystems
A = reshape([-0.1], 1, 1) # creates 1x1 matrix, [-1.0] is a vector, not a matrix
b_on = [5]
mode_off = ConstrainedLinearContinuousSystem(A, p_domain_off)
mode_on  = ConstrainedAffineContinuousSystem(A, b_on, p_domain_on) # TODO Affine is not yet defined on MathematicalSystems

The reset map contains both the reset map and guard as respectively the dynamic and the discrete state constraint set.
Here the reset map is the identity so we use a `ConstraintDiscreteIdentitySystem`.

In [10]:
using MathematicalSystems
const dimension = 1
resetmap_off_to_on = ConstrainedDiscreteIdentitySystem(dimension, guard_off_to_on)
resetmap_on_to_off = ConstrainedDiscreteIdentitySystem(dimension, guard_on_to_off);

We index, say, the mode off as 1 and the mode on as 2 and
we assign the label 1 to the off to on transition and label 2 to the on to off transition.

In [ ]:
modes = [mode_off, mode_on]
resetmaps = [resetmap_off_to_on, resetmap_on_to_off]

The automaton can be constructed as follows:

In [14]:
using HybridSystems
a = LightAutomaton(2)
add_transition!(a, 1, 2, 1)  # off to on
add_transition!(a, 2, 1, 2); # on to off

Depending on the context, we might consider the switching as autonomous or controlled, let's consider it autonomous in this example.

In [15]:
switching = AutonomousSwitching()
switchings = fill(switching, 2)

2-element Array{AutonomousSwitching,1}:
 AutonomousSwitching()
 AutonomousSwitching()

We can now create our hybrid automata by combining all our elements.

In [ ]:
hs = HybridSystem(a, modes, resetmaps, switchings)